Install by running `pip install maturin` 

Build by running ` maturin develop --release` which will build and install the wheel to your env

In [ ]:
from time import time
import numpy as np
import pyvips as pv
#import memray
import histomicstk as htk
from histomicstk.saliency.tissue_detection import get_tissue_mask
import pca_deconv
# %load_ext memray

In [ ]:
test_pv = pv.Image.new_from_file("Some Reasonably Sized WSI Image") / 255
# test_pv = test_pv.resize(0.5, kernel="linear")
test = test_pv.numpy()
mask = get_tissue_mask(test, sigma=1)[-1]
print(test.shape, test.dtype, test.min(), test.max())
test = test.astype(np.float64) * 255 + 1
print(test.shape, test.dtype, test.min(), test.max())
print(mask.shape, mask.dtype, mask.min(), mask.max())
# mask=None

In [ ]:
# %%memray_flamegraph --native
start = time()
rs_pca = pca_deconv.py_rgb_separate_stains_macenko_pca(test, None, 16, .01, .99, mask)
print("Rust PCA took", time() - start, "seconds")

In [ ]:
# %%memray_flamegraph --native
start = time()
htk_pca = htk.preprocessing.color_deconvolution.rgb_separate_stains_macenko_pca(test - 1, None, mask_out=mask)
print("HTK PCA took", time() - start, "seconds")

In [ ]:
print("rust: \n", rs_pca)
print("htk: \n", htk_pca)

In [ ]:
np.save("test_image.npy", test)

In [ ]:
# use matplotlib to visualize the two different PCA results, showing the the three channels of both methods independently
import matplotlib.pyplot as plt
htk_stains = htk.preprocessing.color_deconvolution.color_deconvolution(test, htk_pca).Stains
rs_stains = htk.preprocessing.color_deconvolution.color_deconvolution(test, rs_pca).Stains
fig, axs = plt.subplots(2, 3, figsize=(15, 10))
axs[0, 0].imshow(htk_stains[:, :, 0], cmap='gray')
axs[0, 0].set_title('HTK Stain 1')
axs[0, 1].imshow(htk_stains[:, :, 1], cmap='gray')
axs[0, 1].set_title('HTK Stain 2')
axs[0, 2].imshow(htk_stains[:, :, 2], cmap='gray')
axs[0, 2].set_title('HTK Stain 3')

axs[1, 0].imshow(rs_stains[:, :, 0], cmap='gray')
axs[1, 0].set_title('Rust Stain 1')
axs[1, 1].imshow(rs_stains[:, :, 1], cmap='gray')
axs[1, 1].set_title('Rust Stain 2')
axs[1, 2].imshow(rs_stains[:, :, 2], cmap='gray')
axs[1, 2].set_title('Rust Stain 3')

In [ ]:
# visualize each channel of each deconv, but just for a tile at the center of the image:
tile_size = 1024
center_x, center_y = test.shape[1] // 2 + tile_size, test.shape[0] // 2 + tile_size
start_x = center_x - tile_size // 2
start_y = center_y - tile_size // 2

htk_tile = htk_stains[start_y:start_y + tile_size, start_x:start_x + tile_size, :]
rs_tile = rs_stains[start_y:start_y + tile_size, start_x:start_x + tile_size, :]
orig_image_tile = test[start_y:start_y + tile_size, start_x:start_x + tile_size, :]
mask_tile = mask[start_y:start_y + tile_size, start_x:start_x + tile_size]

fig, axs = plt.subplots(2, 4, figsize=(15, 10))
axs[0, 0].imshow(htk_tile[:, :, 0], cmap='gray')
axs[0, 0].set_title('HTK Tile Stain 1')
axs[0, 1].imshow(htk_tile[:, :, 1], cmap='gray')
axs[0, 1].set_title('HTK Tile Stain 2')
axs[0, 2].imshow(htk_tile[:, :, 2], cmap='gray')
axs[0, 2].set_title('HTK Tile Stain 3')
axs[0, 3].imshow(orig_image_tile)
axs[0, 3].set_title("Original Image")

axs[1, 0].imshow(rs_tile[:, :, 0], cmap='gray')
axs[1, 0].set_title('Rust Tile Stain 1')
axs[1, 1].imshow(rs_tile[:, :, 1], cmap='gray')
axs[1, 1].set_title('Rust Tile Stain 2')
axs[1, 2].imshow(rs_tile[:, :, 2], cmap='gray')
axs[1, 2].set_title('Rust Tile Stain 3')
axs[1, 3].imshow(mask_tile)
axs[1, 3].set_title("Mask")

plt.tight_layout()
plt.show()

In [ ]:
print(rs_pca[0].dtype, rs_pca[0].min(), rs_pca[0].max())
print(htk_pca[0].dtype, htk_pca[0].min(), htk_pca[0].max())
print(rs_pca[1].dtype, rs_pca[1].min(), rs_pca[1].max())
print(htk_pca[1].dtype, htk_pca[1].min(), htk_pca[1].max())
print(rs_pca[2].dtype, rs_pca[2].min(), rs_pca[2].max())
print(htk_pca[2].dtype, htk_pca[2].min(), htk_pca[2].max())